In [ ]:
import affine, concurrent, cStringIO, glob, IPython, json, os, PIL, sys, thread, time, traceback


In [ ]:
class SimpleProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleProcessPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleProcessPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleProcessPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleProcessPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleProcessPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleProcessPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

class Stopwatch:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, type, value, traceback):
        sys.stdout.write('%s took %.1f seconds\n' % (self.name, time.time() - self.start))


In [ ]:
def create_index():
    c = "CREATE INDEX IF NOT EXISTS wdpa_species_counts_idx ON wdpa_species_counts (wdpa_gid);"
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd    

def vacuum_table():
    c = "VACUUM ANALYZE wdpa_species_counts;"
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd   
    

def select_into():
    c = """
    SELECT 
      wdpa_gid, 
      sum(all_count) as all_count,
      sum(cr_count) as cr_count,
      sum(cr_areas) as cr_areas,
      sum(en_count) as en_count,
      sum(en_areas) as en_areas,
      sum(vu_count) as vu_count,
      sum(vu_areas) as vu_areas,
      sum(nt_count) as nt_count,
      sum(nt_areas) as nt_areas,
      sum(lc_count) as lc_count,
      sum(lc_areas) as lc_areas      
    INTO wdpa_species_counts
    FROM wdpa_raw_counts
    GROUP BY wdpa_gid 
    ORDER BY wdpa_gid
    """
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd



In [ ]:
select_counts()

In [ ]:
select_into()

In [ ]:
create_index()

In [ ]:
vacuum_table()